# Yandex.Music

There are a lot of myths when we try to compare Moscow and Saint Petersburg, for example:
 * Moscow is a metropolis subject to the rigit rhythm of the working week;
 * Saint Peterburg is a cultural capital with its own preferences.


Using Yandex.Music data, we will compare the behavior of users in the two capitals.

**The purpose of the research** is test three hypotheses:
1. User activity depends on the day of the week. Moreover, in Moscow and St. Petersburg, this manifests in different ways.
2. On Monday morning, certain genres dominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday nights are dominated by different genres, depending on the city. 
3. People in Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg people prefer russian rap.

**The research progress**
 
The research will take place in three stages:
 1. Data review.
 2. Data preprocessing.
 3. Hypothesis testing.



## Data review

Get a first notion about Yandex.Music data.




In [1]:
# import pandas
import pandas as pd  

In [2]:
# reading data file and saving in df
df = pd.read_csv('...csv')  

In [3]:
# getting the first 10 rows of the table df
display(df.head(10))  

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# getting general information about the date in the table df
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


So the table has seven columns. Data type in all columns — `object`.

According to data documentation:
* `userID` — user ID;
* `Track` — track title;  
* `artist` — artist name;
* `genre` — genre name;
* `City` — user's city;
* `time` — listening start time;
* `Day` — day of the week.

There are three style violations in the column headings:
1. Lowercase letters are combined with uppercase.
2. There are missing values.
3. The first column 'userID', must be written in 'snake case': 'user_id'



The number of values in the columns varies. So there are missing values in the data.


**Conclusions**

Each line of the table contains data about the track users have listened to. Some of the columns describe the composition itself: title, artist and genre. The rest of the data tells about the user: what city user is from, when user listened to music. 

Preliminarily, it can be argued that there is enough data to test hypotheses. But there are missing values, and there is a bad style in the names of some columns.

To move forward, we need to fix problems in the data.

## Data preprocessing

Let's fix the style in the column headings, eliminate missing values. Then check the data for duplicates.

### Style in the column headings


In [5]:
# list of table column names df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Let's bring the column names into good style:
* write a few words in the title in «snake_register»,
* make all characters lowercase,
* remove all missing values.

To do this, rename the columns like this:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
# renaming columns
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

Let's check the result. To do this, once again display the names of the columns:

In [7]:
# check results
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Data missing values

First, let's calculate how many missing values ​​are in the table. There are two `pandas` methods for this :

In [8]:
# calculating missing values
print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Not all missing values affect the research. So in `track` and `artist` missing values are not important for our project. It suffices to replace them with explicit notation.

But missing values in  `genre` may interfere with the comparison of musical preferences in Moscow and St. Petersburg. In practice, it would be correct to determine the cause of the missing values and restore the data. This option is not available in the curriculum. So we have to:
* fill in these missing values with explicit notation,
* estimate how these missing values will damage the calculations. 

So let's replace the missing values in the columns `track`, `artist` and `genre` with `'unknown'`. To do this, let's create a list `columns_to_replace`, loop through its elements with using  `for` and for each column, perform the replacement of missing values:

In [9]:
# iterate through the column names in a loop and replace the missing values with 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for elem in columns_to_replace:
    df[elem] = df[elem].fillna('unknown')

Make sure there are no missing values in the table. To do this, once again calculate the missing values.

In [10]:
# calculating the missing values
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicates

Let's calculate the explicit duplicates in the table:

In [11]:
# calculating the explicit duplicates
df.duplicated().sum()

3826

In [12]:
# removal of explicit duplicates (with the removal of old indexes and the formation of new ones)
df = df.drop_duplicates().reset_index(drop=True)

Once again, we will count the explicit duplicates in the table and make sure that we completely delete all of it:

In [13]:
# check foe missing explicit duplicates
df.duplicated().sum()

0

Now let's delete the implicit duplicates in the column `genre`. For example, the name of the same genre can be spelled slightly differently. Such errors will also affect the result of the research.

Let's display a list of unique genre names, sorted alphabetically. For this:
* extract the desired dataframe column, 
* apply the sort method to it,
* for a sorted column, call a method that will return unique values from the column.

In [14]:
# view unique genre names
df_genre = df['genre']
df_sorted = df_genre.sort_values()
df_sorted = df_sorted.unique()
print(df_sorted)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 


Let's go through the list and find implicit duplicates of the name `hiphop`. These may be misspelled titles or alternative titles in the same genre.

We can see the following implicit duplicates:
* *hip*,
* *hop*,
* *hip-hop*.

To clear the table from it, we write a function `replace_wrong_genres()` with two parameters: 
* `wrong_genres` — duplicates list,
* `correct_genre` — string with correct value.

The function should fix the column `genre` in the table `df`: replace each value from the list `wrong_genres` with value from `correct_genre`.

In [15]:
# the function to replace implicit duplicates
def replace_wrong_genre(data, column, wrong_genres, correct_genres):
    for wrong_genre in wrong_genres:
        data[column] = data[column].replace(wrong_genre, correct_genres)
    return data    

Call the fanction `replace_wrong_genres()` and pass it such arguments so that it eliminates implicit duplicates: instead of `hip`, `hop` and `hip-hop` tha table should have the value `hiphop`:

In [16]:
# eliminate implicit hip-hop duplicates
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genres = 'hiphop'
df = replace_wrong_genre(df, 'genre', wrong_genres, correct_genres)

Check that the wrong column values have been replaced:

*   hip
*   hop
*   hip-hop

Output a sorted list of unique column values `genre`:

In [17]:
# checking for implicit duplicates
df_genre = df['genre']
df_sorted = df_genre.sort_values()
df_sorted = df_sorted.unique()
print(df_sorted)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Conclusions**

Data preprocessing found three problems in the data:

- header style violations,
- missing values,
- explicit and implicit duplicates.

We've fixed the headers to make the table easier to work with. Without duplicates, the research will become more accurate.

Missing values we replaced with `'unknown'`. It remains to be seen whether the missing values in the column `genre` will harm the research .

Now we can move on to hypothesis testing. 

## Hypothesis testing

### Comparation of user preferences in two capitals

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Let's check this hypothesis on the data on three days of the week - Monday, Wednesday and Friday. For this we need to:

* separate users of Moscow and St. Petersburg
* compare how many tracks each group of users listened to on Monday, Wednesday and Friday.


For training, we first perform each of the calculations separately. 

Let's evaluate the activity of users in each city. Let's group the data by city and count the plays in each group.



In [18]:
# counting plays in each city
print(df.groupby('city')['city'].count())

city
Moscow              42741
Saint-Petersburg    18512
Name: city, dtype: int64


There are more listenings in Moscow than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.

Now let's group the data by day of the week and count the plays on Monday, Wednesday, and Friday. It is necessary to take into account that the data contains information only about listening only for these days.


In [19]:
# listening count for each of the three days
print(df.groupby('day')['day'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64


On the average, users from the two cities are less active on Wednesdays. But everything can change if we consider each city separately. 

Now write a function that will combine the following calculations.

Create a function `number_tracks()`, which will calculate the listenings for a given day and city. It needs two parameters:
* day of the week,
* city.

In the function, we save in a variable the rows of the source table that have the parameters:
  * for the column `day` will be parametr `day`,
  * for the column `city` will be parametr `city`.

To do this, apply sequential filtering with logical indexing.

Then we count the values in the `user_id` column of the resulting table. The result will be saved in a new variable. Return this variable from the function.

In [20]:
# <creating the function number_tracks()>
# Declaring a function with two parameters: day, city.
# The track_list variable saves those rows of the df table for which 
# the value in the 'day' column is equal to the day parameter and at the same time the value
# in the 'city' column is equal to the city parameter (use sequential filtering with logical indexing).
# The variable track_list_count saves the number of column values 'user_id',
# calculated by the count() method for the table track_list.
# The function returns a number which is the value of track_list_count.

# Function for counting listenings for a specific city and day.
# With using sequential filtering with logical indexing
# it will first get the rows with the desired day from the source table,
# then from the result it will filter the rows with the desired city,
# the count() method will count the number of values in the user_id column. 
# The function will return this number as a result.
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Let's call `number_tracks()` six times, changing the value of the parameters so that we get data for each city on each of the three days.

In [21]:
# amount of listenings in Moscow on Mondays
track_list_moscow_monday = number_tracks('Monday', 'Moscow')

In [22]:
# amount of listenings in Saint Petersburg on Mondays
track_list_spb_monday = number_tracks('Monday', 'Saint-Petersburg')

In [23]:
# amount of listenings in Moscow on Wednesdays
track_list_moscow_wednesday = number_tracks('Wednesday', 'Moscow')

In [24]:
# amount of listenings in Saint Petersburg on Wednesdays
track_list_spb_wednesday = number_tracks('Wednesday', 'Saint-Petersburg')

In [25]:
# amount of listenings in Moscow on Fridays
track_list_moscow_friday = number_tracks('Friday', 'Moscow')

In [26]:
# amount of listenings in Saint Petersburg on Fridays
track_list_spb_friday = number_tracks('Friday', 'Saint-Petersburg')

Let's create a table using the `pd.DataFrame` constructor, where
* column names are `['city', 'monday', 'wednesday', 'friday']`;
* values are the results which we got with `number_tracks`.

In [27]:
# Results table
columns = ['city', 'monday', 'wednesday', 'friday']
data = [
    ['Moscow', track_list_moscow_monday, track_list_moscow_wednesday, track_list_moscow_friday],
    ['Saint-Petersburg', track_list_spb_monday, track_list_spb_wednesday, track_list_spb_friday]
]
track_list_city_day = pd.DataFrame(data=data, columns=columns)
display(track_list_city_day)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Conclusions**


Data shows the difference in user preferences:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, people listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally less than in the Wednesday.

So the data support the first hypothesis.

### Listenings at the beginning and end of the week

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday nights are dominated by different genres, depending on the city.

Save tables with data in two variables:
* data in Moscow are in a variable `moscow_general`;
* data in Saint Petersburg are in a variable `spb_general`.

In [28]:
# getting moscow_general table from those df table rows, 
# for which the value in the 'city' column is 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [29]:
# getting spb_general table from those df table rows,
# for which the value in the 'city' column is 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Create a `genre_weekday()` function with four parameters:
* table (dataframe) with data,
* day of the week,
* initial timestamp in 'hh:mm' format, 
* last timestamp in 'hh:mm' format.


The function should return information about the top 10 genres of those tracks that were listened to on the specified day, in the interval between two timestamps.

In [30]:
# Declaring genre_weekday() function with table, day, time1, time2 parameters,
# which returns information about the most popular genres on a given day at a given time:
# 1) the genre_df variable saves those rows of the passed dataframe table, for which at the same time:
#    - the value in the day column is equal to the value of the day argument
#    - the value in the time column is greater than the value of the time1 argument
#    - the value in the time column is less than the value of the time2 argument
#    Use the sequential filtering with logical indexing.
# 2) group genre_df dataframe by genre column, take one of its
#    columns and use the count() method to calculate the number of records for each of
#    present genres, write the resulting Series to the genre_df_count variable
# 3) sort genre_df_count in descending order of occurrence and store in the variable genre_df_sorted
# 4) return a Series of the first 10 genre_df_sorted values, these will be the top 10
#    popular genres (on a specified day, at a specified time)
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)

Compare the results of the `genre_weekday()` function for Moscow and St. Petersburg on Monday morning (from 7:00 to 11:00) and Friday evening (from 17:00 to 23:00):

In [31]:
# function call for Monday morning in Moscow (instead of df we use moscow_general table)
# time objects are strings and are compared as strings
genre_moscow_monday = genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
print(genre_moscow_monday)

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64


In [32]:
# function call for Monday morning in St. Petersburg (instead of df we use spb_general table)
genre_spb_monday = genre_weekday(spb_general, 'Monday', '07:00', '11:00')
print(genre_spb_monday)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64


In [33]:
# function call for Friday evening in Moscow
genre_moscow_friday = genre_weekday(moscow_general, 'Friday', '17:00', '23:00')
print(genre_moscow_friday)

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64


In [34]:
# function call for Friday evening in St. Petersburg
genre_spb_friday = genre_weekday(spb_general, 'Friday', '17:00', '23:00')
print(genre_spb_friday)

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64


**Conclusions**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg people listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical music.

2. There were so many missing values in Moscow that the value `'unknown'` took the tenth place among the most popular genres. This means that missing values occupy a significant share in the data and threaten the reliability of the research.

Friday night does not change this conclusions. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, people listen to Russian popular music more often, in St. Petersburg people listen to jazz.

However, missing values in the data cast doubt on this result. There are so many of them in Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg

Let's put forward a hypothesis: Petersburg is the capital of rap, the music of this genre is listened to there more often than in Moscow. And Moscow is a city of contrasts, in which, nevertheless, pop music prevails.

Let's group the `moscow_general` table by genre and count the listens of tracks of each genre using the `count()` method. Then we sort the result in descending order and save it in the `moscow_genres` table.

In [35]:
# in one line: grouping the moscow_general table by the 'genre' column, 
# counting the number of 'genre' values in this grouping using the count() method, 
# sorting the resulting Series in descending order and saving it to moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Display the first ten lines of `moscow_genres`:

In [36]:
# view the first 10 lines of moscow_genres
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


Now let's repeat the same for St. Petersburg.

Let's group the `spb_general` table by genre. Let's count listening to tracks of each genre. Sort the result in descending order and save it in the `spb_genres` table:


In [37]:
# in one line: group spb_general table by 'genre' column, 
# counting the number of 'genre' values in this grouping using the count() method, 
# sorting the resulting Series in descending order and saving it to spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Display the first ten lines of `spb_genres`:

In [38]:
# view the first 10 lines of spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Conclusions**

The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a similar genre. It is a Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg. 


## Research results

We tested three hypotheses and found:

1. The day of the week affects the activity of users in Moscow and St. Petersburg differently. 

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week in Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* people in Moscow listen to world music,
* people in Saint Petersburg listen to jazz and classical.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for missimg values in the data.

3. The tastes of users in Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the majority of users.
